In [1]:
import torch
from torch import nn
from torch.nn import functional as F
import math

In [7]:
class TokenEmbedding(nn.Embedding):
    def __init__(self,vacab_size,d_model):
        super().__init__(vacab_size,d_model,padding_idx=1)

In [8]:
class PositionalEmbedding(nn.Module):
    def __init__(self,d_model,max_len,device):
        super().__init__()
        self.encoding=torch.zeros(max_len,d_model,deivice=device)
        self.encoding.requires_grad=False
        pos=torch.arrange(0,max_len,device=device)
        pos=pos.float().unsqueeze(dim=1)
        _2i=torch.arrange(0,d_model,step=2,device=device).float()
        self.encoding[:,0::2]=torch.sin(pos/(10000**(_2i/d_model)))
        self.encoding[:,1::2]=torch.cos(pos/(10000**(_2i/d_model)))
    def forward(self,x):
        batch_size,seq_len=x.size()
        return self.encoding[:seq_len,:]

In [9]:
class TransformerEmbedding(nn.Module):
    def __init__(self,vacab_size,d_model,max_len,dropout,device):
        super().__init__()
        self.token_embedding=TokenEmbedding(vacab_size,d_model)
        self.position_embedding=PositionalEmbedding(d_model,max_len,device)
        self.dropout=nn.Dropout(p=dropout)
        self.d_model=d_model
    def forward(self,x):
        tok_emb=self.token_embedding(x)
        pos_emb=self.position_embedding(x)
        return self.dropout(tok_emb+pos_emb)